#  Visualize points as heatmap

In particular situations when the number of points on the map is excessive their visualization can become problematic. Essentially
the points start to overlap and crods the map up to the point where it becomes unreadable. A possible solution is representated by
vizualizing the points as a **heatmap**, focusing on the density or intensity of data points across a geographic area. Instead of showing individual points, a heatmap uses color gradients to show where data points are concentrated, allowing patterns and trends to be visually interpreted more easily.


>  a heatmap **should not** be used to represent anything else than the density ofd the points. While a heatmap appears similar to a surface it does not share any of characteritics. 



## Imports

alter sys.path so we can import utils module

In [1]:
import sys
import os
import ipywidgets as widgets
# Add parent directory to sys.path
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import utils

In [3]:
import leafmap.maplibregl as leafmap

## Data

We are going to vizualize a point layer representing building from east of Bangladesh. Geohub features [REST API](https://dev.undpgeohub.org/api)
and as a result it is possible to searh for data usingh the [/datasets](https://dev.undpgeohub.org/api/datasets) endpoint. The [utils module](../utils.py) contains a search function that takes a query expression or workd and returns a dictionary where the key is the dataset id and value is a list of links.

In [4]:
query='bangladesh buildings'
results = utils.search_datasets(query=query)

In [5]:

ds_id, links = next(iter(results.items()))
pmtiles_url = [e['href'] for e in links if e['rel'] == 'download'].pop()


## Visualization

leafmap is equipped with some util functions that allow us to inspect the PMTiles file

In [6]:
metadata = leafmap.pmtiles_metadata(pmtiles_url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")


layer names: ['bldgsc']
bounds: [90.8483229, 23.3422379, 91.063028, 23.510859]


## Attributes or properties

Vector tiles are very efficient with compressing attributes or properties. The medata variable crteated above can be used to explore/dicover
the properties of attributes that existy in the data file.


In [7]:
import json
#print(json.dumps(metadata, indent=2))
fields = metadata['vector_layers'][0]['fields']
print(json.dumps(fields, indent=4))
attrs = metadata['tilestats']['layers'][0]['attributes']
print('\n')


{
    "area_in_meters": "Number",
    "bf_source": "String",
    "boundary_id": "Mixed",
    "confidence": "Number",
    "country_iso": "String",
    "geohash": "String",
    "s2_id": "Mixed"
}




The best chance to assign a color to every point is to inspoect the values of a string type attribute or property like **bf_source**


In [8]:
print('\n')
for e in attrs:
    if e['attribute'] == 'bf_source':
        print(f'bf_source attribute values are {e["values"]}\n')




bf_source attribute values are ['google', 'microsoft', 'osm']



We will assign a scperific color to every points based on the values of the **bf_source** attribute

In [9]:
m = leafmap.Map(center=[0, 20], zoom=2, height="600px", 
                style='https://dev.undpgeohub.org/api/mapstyle/style.json'
               )

# create a maplibre style object as a python dictionary 
simple_point_style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + pmtiles_url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "bbuildings",
            "source": "example_source",
            "source-layer": "bldgsc",
            #"maxzoom": 9,
            "type": "heatmap",
            
            "paint": {
                    'heatmap-weight': [
                      'interpolate', ['linear'],
                      ['get', 'confidence'], 0, 0, 0.1, 0.1, 0.2, 0.2, 0.3, 0.3,
                      0.5, 0.5, 1, 1,
                    ],
                    'heatmap-intensity': [
                      'interpolate', ['linear'],
                      ['zoom'], 0, 1, 13, 3
                    ],
                    'heatmap-color': [
                      'interpolate', ['linear'],
                      ['heatmap-density'], 
                        0,
                        "rgba(33,102,172,0)",
                        0.2,
                        "rgb(103,169,207)",
                        0.4,
                        "rgb(209,229,240)",
                        0.6,
                        "rgb(253,219,199)",
                        0.8,
                        "rgb(239,138,98)",
                        1,
                        "rgb(178,24,43)",
                    ],
                    'heatmap-radius': [
                      'interpolate', ['linear'],
                      ['get', 'population'], 1000, 1, 1000000, 20, 10000000, 30
                    ],
                    'heatmap-opacity': [
                      'interpolate', ['linear'],
                      ['zoom'], 7, 1, 9, 0
                    ]
                  },
        }
    ]
}
m.add_pmtiles(
    pmtiles_url,
    style=simple_point_style,
    visible=True,
    opacity=1.0,
    tooltip=True,
)

m.layer_interact()



TraitError: The 'value' trait of a FloatSlider instance expected a float, not the list ['interpolate', ['linear'], ['zoom'], 7, 1, 9, 0].

In [ ]:
tab_contents = ['P0', 'P1', 'P2', 'P3', 'P4']
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = [str(i) for i in range(len(children))]
tab

In [ ]:
m

In [ ]:
m.fit_bounds(metadata['bounds'])